https://github.com/truera/trulens/blob/main/trulens_eval/examples/expositional/frameworks/llama_index/llama_index_retrievalquality.ipynb

基本按照原样编写的，但最终报错

In [10]:
%%time
%%capture

!pip install llama-index-embeddings-langchain

CPU times: user 11.3 ms, sys: 12.1 ms, total: 23.5 ms
Wall time: 1.93 s


In [1]:
from trulens_eval import Tru

tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [2]:
%%time

from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.chunk_size = 128
Settings.chunk_overlap = 16

Settings.llm = OpenAILike(
    model="qwen2", 
    api_base="http://monkey:11434/v1", 
    api_key="ollama",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model =OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://monkey:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

CPU times: user 23.7 ms, sys: 4.31 ms, total: 28 ms
Wall time: 27.2 ms


In [3]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model = HuggingFaceEmbeddings(
    model_name="/models/paraphrase-multilingual-MiniLM-L12-v2"
)

In [11]:
from llama_index.core import VectorStoreIndex
from llama_index.legacy import ServiceContext
from llama_index.readers.web import SimpleWebPageReader

documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["https://paulgraham.com/worked.html"]
)

# service_context = ServiceContext.from_defaults(embed_model=embed_model,llm=Settings.llm)

Settings.embed_model =embed_model

index = VectorStoreIndex.from_documents(
    documents,
    # service_context=service_context
)

query_engine = index.as_query_engine(top_k=5)

In [12]:
response = query_engine.query("What did the author do growing up?")
print(response)

Growing up, the author focused on two main activities outside of school: writing short stories as a beginning writer and programming. They did not write essays but instead worked on crafting short stories that lacked plot development, featuring characters with strong emotions they believed made them deep. Additionally, they enjoyed programming but initially had no plans to study it in college, opting instead for philosophy which seemed more powerful and concerned with ultimate truths according to their naive high school perspective.


In [13]:
import os
from trulens_eval.feedback.provider import OpenAI

os.environ["OPENAI_API_KEY"] = "sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750"
os.environ["OPENAI_API_BASE"] = "https://ape:3000/v1"
provider = OpenAI(
    model_engine="qwen2"
)

In [14]:
import numpy as np
from trulens_eval import Feedback
from trulens_eval import TruLlama

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(provider.context_relevance)
    .on_input()
    .on(TruLlama.select_source_nodes().node.text)
    .aggregate(np.mean)
)

✅ In context_relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In context_relevance, input context will be set to __record__.app.query.rets.source_nodes[:].node.text .


In [15]:
from trulens_eval.feedback import Embeddings

f_embed = Embeddings(embed_model=embed_model)

# f_embed_dist = (
#     Feedback(f_embed.cosine_distance)
#     .on_input()
#     .on(TruLlama.select_source_nodes().node.text)
#     .aggregate(np.mean)
# )

AttributeError: 'Embeddings' object has no attribute '__pydantic_private__'